## DataSet Creation

In [1]:
import zipfile
import os
import json
import librosa
import numpy as np

In [2]:
mcz_files = os.listdir("original_beatmaps")

In [3]:
def load_audio(audio_file):
    x , sr = librosa.load(audio_file, sr=20000)
    return x, sr

In [4]:
def hcf(x, y):
   """该函数返回两个数的最大公约数"""
 
   # 获取最小值
   if x > y:
       smaller = y
   else:
       smaller = x
   _hcf = 1
   for i in range(1,smaller + 1):
       if((x % i == 0) and (y % i == 0)):
           _hcf = i
 
   return _hcf

In [5]:
def get_audio_features(x, sr, bpm, position, offset):
    one_beat = 60 / bpm
    beat = position * one_beat / 4 - offset/1000
    
    start = beat
    end = start + one_beat / 8
    
    end2 = start + one_beat / 4
    if start < 0:
        start = 0
    
#     print(start, end)
    start_index = int(sr * start)
    end_index = int(sr * end)
    
#     start_index2 = int(sr * start2)
    end_index2 = int(sr * end2)
    
    features = []
    mfcc1 = librosa.feature.mfcc(y=x[start_index:end_index], sr=sr, n_mfcc=32)
    mfcc2 = librosa.feature.mfcc(y=x[end_index:end_index2], sr=sr, n_mfcc=32)
    
    features += [float(np.mean(e)) for e in mfcc1]
    features += [float(np.mean(e)) for e in mfcc2]
    
    return features

In [6]:
def get_columns_list(notes):
    columns_list = []
    columns = {
        0: {},
        1: {},
        2: {},
        3: {}
    }

    for note in notes:
        if 'column' in note:
            beat = note['beat'][0]
            sub_beat = note['beat'][1]
            split_count = note['beat'][2]
            if split_count == 8:
                if (len(columns[0]) != 0) and (len(columns[1]) != 0) \
                    and (len(columns[2]) != 0)and (len(columns[3]) != 0):
                    columns_list.append(columns)
                    columns = {0: {}, 1: {}, 2: {}, 3: {}}
                continue
            if split_count != 4:
                if sub_beat == 0:
                    split_count = 4
                else:
                    _hcf = hcf(sub_beat, split_count)
                    sub_beat = int(sub_beat / _hcf)
                    split_count = int(split_count / _hcf)
                if split_count == 2:
                    sub_beat *= 2
                    split_count *= 2
                if split_count == 1:
                    sub_beat *= 4
                    split_count *= 4
                elif split_count != 4:
                    if (len(columns[0]) != 0) and (len(columns[1]) != 0) \
                        and (len(columns[2]) != 0)and (len(columns[3]) != 0):
                        columns_list.append(columns)
                        columns = {0: {}, 1: {}, 2: {}, 3: {}}
                    continue

            position = beat * 4 + sub_beat
            which_col = note["column"]
            if(which_col >= 4):
                continue
            if "endbeat" in note:
                end_position = note["endbeat"][0] * 4 + int(note["endbeat"][1] / note["endbeat"][2] * 4)
                if end_position == position:
                    columns[which_col][position] = 1
                else:
                    for i in range(position, end_position+1):
                        columns[which_col][i] = 2
            else:
                columns[which_col][position] = 1
    return columns_list

In [7]:
def get_columns_min_max(columns):
    _min = 10000000000
    _max = 0
    for col in columns.keys():
        column = columns[col]
        if max(column.keys()) > _max:
            _max = max(column.keys())

        if min(column.keys()) < _min:
            _min = min(column.keys())
    return _min, _max
    

In [8]:
def get_one_data(start, end, columns, bpm, x_, sr, offset):
    # 判断是否有beat
    x0 = []
    y0 = []
    
    # 判断note的键型
    x1 = []
    y1 = []
    
    
     # 判断是否有long_note
    x2 = []
    y2 = []
    
    # 判断long_note的键型
    x3 = []
    y3 = []
    for i in range(start, end):
        audio_features = get_audio_features(x_, sr, bpm, i, offset)
        x0.append(audio_features)
        x2.append(audio_features)
        beat_count = 0
        has_beat = False
        has_ln = False
        long_note_count = 0
        # column 0
        if i in columns[0]:
            if columns[0][i] == 1:
                has_beat = True
                beat_count += 1
            else:
                has_ln = True
                long_note_count += 1
            
        
        # column 1
        if i in columns[1]:
            if columns[1][i] == 1:
                has_beat = True
                beat_count += 2
            else:
                has_ln = True
                long_note_count += 2
            
        # column 2
        if i in columns[2]:
            if columns[2][i] == 1:
                has_beat = True
                beat_count += 2*2 
            else:
                has_ln = True
                long_note_count += 2*2
        
        # column 3
        if i in columns[3]:
            if columns[3][i] == 1:
                has_beat = True
                beat_count += 2*2*2
            else:
                has_ln = True
                long_note_count += 2*2*2
        
        y0.append(int(has_beat))
        
        if has_beat:
            x1.append(audio_features)
            y1.append(beat_count)
            
        y2.append(int(has_ln))
        
        if has_ln:
            x3.append(audio_features)
            y3.append(long_note_count)
        
    return x0, y0, x1, y1, x2, y2, x3, y3

In [9]:
count = 0
X0 = []
Y0 = []
X1 = []
Y1 = []
X2 = []
Y2 = []
X3 = []
Y3 = []

x_ = []
sr = 0
for mcz_file in mcz_files:
    if ".mcz" not in mcz_file:
        continue
    print(count, mcz_file)
    zFile = zipfile.ZipFile("original_beatmaps/" + mcz_file, "r")
    audio_file = ""
    mc_file = ""
    mc_data = {}
    for fileM in zFile.namelist():
        zFile.extract(fileM, './')
        if ".mc" in fileM:
            mc_file = fileM
            data = zFile.read(fileM).decode("utf-8")
            mc_data = json.loads(data)
            print("\t", mc_data["meta"]["version"], mc_data["time"][0], mc_data["note"][-1], "\n")
        elif ".ogg" in fileM:
            audio_file = fileM
        elif ".mp3" in fileM:
            audio_file = fileM
    
    notes = mc_data["note"]
    notes = notes[:len(notes)-1]
    columns_list = get_columns_list(notes)
    bpm = mc_data["time"][0]['bpm']
    if "offset" not in mc_data["note"][-1]:
        offset = 0 
    else:
        offset = mc_data["note"][-1]["offset"]
    
    print(audio_file, bpm, offset)
    
    x_, sr = load_audio(audio_file)
    
    print(len(x_), sr, "\n")
    no_ln_count = 0
    for columns in columns_list:
        _min, _max = get_columns_min_max(columns)
        if (_max - _min) > 40:
            _now = _min
#             print(_now)
            while (_now + 40) < _max:
                x0, y0, x1, y1, x2, y2, x3, y3,  = get_one_data(_now, _now+40, columns, bpm, x_, sr, offset)
                X0.append(x0)
                Y0.append(y0)
                if len(y1) >= 1:    
                    X1.append(x1)
                    Y1.append(y1)
                
                
                if len(y3) > 0:
                    X2.append(x2)
                    Y2.append(y2)
                    X3.append(x3)
                    Y3.append(y3)
                elif (len(y1) >= 1) and (no_ln_count < 15):
                    X2.append(x2)
                    Y2.append(y2)
                    no_ln_count += 1
                
                _now += 38
            
#     break
    count += 1

0 002051846.mcz
	 4K EXHAUST Lv.26 {'beat': [0, 0, 1], 'bpm': 222.00000000000026} {'beat': [0, 0, 1], 'sound': 'Song.ogg', 'vol': 100, 'offset': 142, 'type': 1} 

Song.ogg 222.00000000000026 142
2357834 20000 

1 015708939.mcz
	 4K Master Lv.24 {'beat': [0, 0, 1], 'bpm': 192, 'signature': 4, 'vol': 5} {'beat': [410, 1, 2], 'column': 1} 



KeyError: 4

In [10]:
print(len(X0[0][0]))

64


In [11]:
print(len(X0))

34


In [12]:
print(len(X1))

34


In [13]:
print(len(X2))

34


In [14]:
print(len(X3))

23


In [15]:
with open("dataset.json", "w") as f:
    json.dump({
        "X0": X0,
        "Y0": Y0,
        "X1": X1,
        "Y1": Y1,
        "X2": X2,
        "Y2": Y2,
        "X3": X3,
        "Y3": Y3,
    }, f)

In [16]:
with open("glove/malody.txt", "w") as f:
    for y1 in Y1:
        strs = [str(i) for i in y1]
        line = " ".join(strs)
        print(line)
        f.write(line + "\n")

2 4 2 2 4 4 1 8 1 8 2 4 8 2 4 8 4 2 8 4 2 8
8 2 4 8 2 4 2 4 2 2 4 4 1 2 8 1 2 8 1 2 8 1 2
1 2 8 1 4 1 4 8 2 8 2 3 12 3 4 2
4 2 8 4 1 2 4 8 10 4 2 4 4 2 4 2 4 4 2
4 1 4 3 3 12 12 3 12 3 12 3 12 3 12
3 3 12 4 11 2 12 1 10 4 3 2 13 4 3 8 3 12 3
12 3 12 3 4 11 2 12 1 10 4 3 2 13 4 11 6 3 2 13
13 4 3 8 10 2 13 2 5 8 3 12 3 12 3 2 13 4 10 2
2 5 4 11 2 13 4 11 2 13 4 3 6 12 1 10 2 6 2 9
4 1 4 3 2 5 4 10 2 5 4 11 3 13 12 7 3 14 12 11 11
11 13 13 7 7 14 14 13 10 5 10
10 1 2 4 8 3 12 3 6 12 3 6 12 3 12 6 9
9 2 12 1 4 1 8 2 8 1 12 6 3 12 1 2 4 8 3 6 12
2 8 2 7 4 2 8 4 8 2 8 2 1
1 4 2 3 6 12 1 2 4
4 8 2 4 1 12 6 1 2 4 8 1 2 4 8 3 12 3 12 3
12 3 12 3 12 3 13 6 11 6 4 11 4 3 8 5 2 12 2
3 12 2 9 4 3 8 5 2 13 4 11 2 13 4 3 12 3 8 4
3 12 2 9 4 11 2 13 4 11 2 4 3 12 3 12 3
4 1 12 3 3 6 5 4 2
2 4 8 4 2 4 8 4 2
2 1 2 4 2 1 2 4 8
8 4 2 4 8 4 2 1 4 2 8 4 1 2 4 8 2 4 1 2 8 4 2 1
8 4 2 1 4 2 8 4 3 8 4 3 12 1 2 12 3 5 12 10
10 1 2 4 8 3 8 4 3 12 1 2 12 3 4 1 2 4 2 1 2 4 2 5 10 1 2 12
2 12 3 12 2 1 12 1 2 12 3 

In [17]:
with open("glove/malody2.txt", "w") as f:
    for y3 in Y3:
        strs = [str(i) for i in y3]
        if len(strs) > 0:
            line = " ".join(strs)
            print(line)
            f.write(line + "\n")

1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1
1 1 1 1 1 8 8 8 8 8 1 1 1 1 1 8 8 8 8 8
1 1 1 1 1
12 12 12 5 5 5 10 10 10
12 12 12 12 12
5 5 5 5
8 8 8 8 8 1 1 1 1 1 1 1 8 8 8 8 8 8 8
8 8 1 1 1 1 1 1 1 8 8 8 8 8 8 8
8 8 8 8 8
8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 9 8 8
8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 9 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 8 8 10
8 10 2 6 4 5 1 1 8 8 8 2 2 2 1 1 1
1 1 4 4 4 8 8 8 8 8 8 8 8 8 8 8
8 8 8 2 2 2
2 2 1 1 1 4 4 4 8 8 8 8 8 8 8 8 8
8 8 8 1 1 1
1 1 1 1 1 1 1 1 1 1
1 1 1 4 4 4 8 8 8 2 2 2 1 1 1 1 1 1 1 1 9 8 8 8 8 8 8
8 8 8 8 8 8
12 12 12 12 12
